---
title: Examples of discontinuities
---


In [2]:
# /// pyproject
# run.requirements = [
#   "pyyaml",
#   "pydantic",
#   "pyspedas",
#   "pytplot",
#   "matplotlib",
#   "scienceplots",
# ]
# /

import sys
import yaml

from pydantic import constr, BaseModel
from datetime import datetime, date

from xarray import DataArray
import pyspedas
from pyspedas.cotrans.minvar_matrix_make import minvar_matrix_make
from pyspedas import tvector_rotate
from pyspedas.analysis.tvectot import tvectot

from pytplot import join_vec, split_vec, get_data
from pytplot import tplot, options

import matplotlib.pyplot as plt
import scienceplots

plt.style.use(["science", "nature", "notebook"])
plt.rc("savefig", dpi=300)
# plt.rc('svg', fonttype='none')

In [3]:
class TplotConfig(BaseModel):
    tvar: str
    trans: list[str] = None


class PanelConfig(BaseModel):
    satellite: constr(to_lower=True)
    instrument: constr(to_lower=True)
    start: datetime | date = None
    end: datetime | date = None
    datatype: str = None
    probe: str = None
    tplot: TplotConfig


class OutputConfig(BaseModel):
    filename: str
    format: str = "png"


class Config(BaseModel):
    panels: list[PanelConfig]
    output: dict = None

In [ ]:
file_path = sys.argv[1]
config = yaml.load(open(file_path), Loader=yaml.FullLoader)

_p_config = config["panels"][0]

panel_config = PanelConfig(**_p_config)
tplot_config = panel_config.tplot

mod = getattr(pyspedas, panel_config.satellite)
func = getattr(mod, panel_config.instrument)

tvar = tplot_config.tvar

In [ ]:
args = {
    "trange": [panel_config.start.isoformat(), panel_config.end.isoformat()],
    "time_clip": True,
}

# Conditionally add the 'datatype' and 'probe' argument
if panel_config.datatype is not None:
    args["datatype"] = panel_config.datatype
if panel_config.probe is not None:
    args["probe"] = panel_config.probe

args['varnames'] = [tvar]

loaded_tvars = func(**args)

In [9]:
[r"$B_l$", r"$B_m$", r"$B_n$"][0:2]

['$B_l$', '$B_m$']

In [ ]:
if tplot_config.trans:
    if "slice-3" in tplot_config.trans:
        tvar = join_vec(split_vec(tvar)[:3])
    if "mva" in tplot_config.trans:
        minvar_matrix_make(tvar)
        tvar = tvector_rotate(tvar + "_mva_mat", tvar)[0]
        ysubtitle = "[nT LMN]"
        legend_names = [r"$B_l$", r"$B_m$", r"$B_n$"]

    tvar2plot = tvectot(tvar, join_component=True)
    legend_names = legend_names + [r"$B_{total}$"]

    # tvars_splitted = split_vec(tvar)

    # tvar2plot = join_vec(tvars_splitted[0:2] + [tvars_splitted[-1]])
    # legend_names = legend_names[0:2] + [legend_names[-1]]

    options(tvar2plot, "ytitle", "$B$")
    options(tvar2plot, "ysubtitle", ysubtitle)
    options(tvar2plot, "legend_names", legend_names)

    options(tvar2plot, "thick", 2)
    options(tvar2plot, "char_size", 16)
else:
    tvar2plot = tvar

In [ ]:
if config["output"]:
    output = config["output"]
    path = output.get("path")
    display = output.get("display", False)
    if display:
        tplot(tvar2plot)
    if "png" in output["formats"]:
        tplot(tvar2plot, save_png=path, display=False)
    if "svg" in output["formats"]:
        tplot(tvar2plot, save_svg=path, display=False)
    if "csv" in output["formats"]:
        da: DataArray = get_data(tvar2plot, xarray=True)
        da.to_pandas().to_csv(path + ".csv")